In [ ]:
import requests
import pandas as pd
import json
import numpy as np
import pprint as pp
from time import sleep
from IPython.display import display
from datetime import datetime
import sys
import datetime
import time

import nltk
from nltk.sentiment import SentimentIntensityAnalyzer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from textblob import TextBlob
import pandas as pd
import re
from time import time
import matplotlib.pyplot as plt
import seaborn as sns

from IPython.display import clear_output

braceDF = pd.read_csv(r'brace_sentiment.csv', index_col='idx', header=0)
lastCommentProcessed = braceDF.index.values.max()
nextComment = int(lastCommentProcessed)+ 1
print(nextComment)

In [ ]:
sia = SentimentIntensityAnalyzer()
stopwords = nltk.corpus.stopwords.words("english")
stop_words = set(stopwords)
stop_words.add('atf')
stop_words.add('rule 2021R')
stop_words.add('2021R')
stop_words.add('batf')
stop_words.add('stabilizing')
stop_words.add('brace')
stop_words.add('rule change')
stop_words.add('proposed')
stop_words.add('rule')


#                       , 'felon'
#                       , 'felons'
#                       , 'arbitrary'
#                       , 'arbitrarily'
#                       , 'laws congress'
#                       , 'violates'
#                       , 'nfa'
#                       , 'congress'
#                       , 'subjective'
#                       , 'do not support'
#                       , 'affront'
#                       , 'has no authority'
#                       , 'violating'
#                       , 'slippery slope'
#                       , 'unnecessary background checks'

oppositionIndicator= [  'unconstitutional'
                      , 'oppose'
                      , 'withdraw'
                      , 'rescind'
                      , 'violate'
                      , 'second amendment'
                      , 'shall not be infringed'
                      , 'infringement'
                      , 'infringe'
                      , 'infringed'
                      , 'infringing'
                      , 'we will not comply'
                      , 'infringed unconstitutional'
                      , 'tyranny'
                      , 'tyrant'
                      , 'tyrants'
                      , 'do not agree'
                      , 'i am against'
                      , 'i am firmly against this'
                      , 'against these changes'
                      , 'does nothing but create felons'
                      , 'making the laws more confusing'
                      , 'immediately withdraw'
                      , 'please do not'
                      , 'slap in the face to gun owners'
                      , '2nd amendment'
                      , 'Second amendment'
                      , 'cant just change law'
                      , 'do not have the authority'
                      , 'do not change'
                      , 'stop taking our'
                      , 'please do not'
                      , 'flagrant attempt'
                      , 'i would not like'
                      , 'an armed society is a polite society'
                      , 'not support this'
                      , 'against the constitution'
                      , 'red tape'
                      , 'stop trying to regulate'
                      , 'corrupt government'
                      , 'punitive tax on firearms'
                      , 'pistol brace is not a stock'
                      , 'would like to see lawmakers change the definition'
                      , 'non-existent issue'
                      , 'gun ban without legislation'
                      , 'will do nothing'
                      , 'overreach'
                      , 'is just wrong'
                      , 'abuse of power'
                      , 'people with disabilities'
                      , 'disabled people'
                      , 'circumvention of our legislative process'
                      , 'meaninglessly bureaucratic'
                      , 'braces are designed'
                      , 'no evidence'
                      , 'criminalizing citizens'
                      , 'too much interpretation'
                      , 'no way convert pistols to rifles'
                      , 'stop changing the rules'
                      , 'i do not support'
                      , 'i can not supoort'
                      , 'not support'
                      , 'i dont agree'
                      , 'i object'
                      , 'i do not believe'
                      , 'comply'
                      , 'will do nothing'
                      , 'do not make any sense'
                      , 'do not approve'
                      , 'I do not believe'
                      , 'reconsider this proposed change'
                      , 'restrict the rights'
                      , 'really poorly thought out'
                      , 'felons overnight'
                      , 'instant felons'
                      , 'dont make millions of pistol-owners into felons'
                      , 'it is a terrible rule and its disposition should be to not enact it'
                      , 'i have narcolepsy with a complication called cataplexy'
                      , 'not make millions of hard working Americans'
                      , 'amend the nfa to deregulate short barreled rifles'
                      , 'determined to be legal'
                      , 'not pursue this rule change'
                      , 'violating'
                      , 'unelected'
                      , 'enjoy shooting firearms'
                      , 'braces do not make a pistol a rifle'
                      , 'i do not think this will accomplish anything'
                      , 'rewrite the law in an arbitrary'
                      , 'the atf has no business'
                      , 'miller'
                      , 'heller'
                      , 'dangerous precident'
                      , 'hope this proposal is reconsidered'
                      , 'are not stocks'
                      , 'millions of law abiding'
                      , 'abandon the proposed rule change'
                      , 'not support this'
                      , 'against the constitution'
                      , 'gross over reach'
                      , 'overreach'
                      , 'will accomplish nothing'
                      , 'stabilizing braces are common'
                      , 'rulemaking discriminates against disabled persons'
                    ]

supportIndicator = [  "they are a criminal's weapon of choice"
                    , "i am writing in support of proposed rule"
                    , "i am writing in support of the"
                    , "this proposed rule is a win for public safety"
                    , "i support this proposed rule"
                    , "i support the proposal"
                    , "and urge the atf to adopt it"
                    , "im urging the atf to protect public safety by adopting this rule"
                    , "protect the public by implementing rule"
                    , "i support rule"
                    , "emerged as a criminals weapon of choice"
                    , "a win for public safety"
                    , "intentionally exploited this narrow exception"
                   ]

def cleanhtml(raw_html):
  cleanr = re.compile('<.*?>|&([a-z0-9]+|#[0-9]{1,6}|#x[0-9a-f]{1,6});')
  cleantext = re.sub(cleanr, '', str(raw_html))
  return cleantext

def sentimentPosition(score):
    position=''
    if score == 0:
        position = 'neutral'
    elif score > 0:
        position = 'support'
    else:
        position = 'oppose'
    return position

def getSentiments(inComment):
    comment = inComment
    comment_tokens = word_tokenize(comment)#[3:240])))
    filtered_sentence = []
    for w in comment_tokens:
        w = str(w).lower()
        if w not in stop_words:
            filtered_sentence.append(w)
#             for word in w.split(' '):
#                 allWords.append(word)
    filtCom = ' '.join(filtered_sentence)
    nltkSentiment = sia.polarity_scores(str(filtCom))['compound']

    flipSent = False
    for senti in oppositionIndicator:
        if nltkSentiment >= 0:
            if senti in str(comment).lower():
#                 print('flipped to oppose')
                flipSent = True
                continue
    for ind in supportIndicator:
        if nltkSentiment < 0:
            if ind.lower() in str(comment).lower():
#                 print('flipped to support')
                flipSent = True
                continue
    
    if flipSent:
        if nltkSentiment == 0:
            nltkSentiment = nltkSentiment-.01
        else:
            nltkSentiment = nltkSentiment*(-1)

    return nltkSentiment

In [ ]:
# ##################### REGENERATE ENTIRE BRACE SENTIMENT CSV FROM COMMENTS CSV ############
# bracesDF = pd.read_csv(r'brace_comment.csv', index_col='idx', header=0)

# bracesDF['comment']= bracesDF['comment'].apply(cleanhtml)
# bracesDF['nltkSentiment'] = bracesDF['comment'].apply(getSentiments)
# bracesDF['sentimentPosition'] = bracesDF['nltkSentiment'].apply(sentimentPosition)
# bracesDF.to_csv(r'brace_sentiment.csv')

In [ ]:
################ BRACES DL AND INSERT 1 COMMENT AT A TIME IN BOTH CSVs ##########

apiKey= #[REGS.GOV API KEY GOES HERE]

def DLandScore(startCommentNumber):
    print('this cycle began: ' +  datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"))
    lastCommentProcessed = startCommentNumber
    for i in range(startCommentNumber,250000):
        clear_output(wait=True)
        lastCommentProcessed = i
#     for i in range(165434,250000):
        sleep(1.8)
#         allWords = []
        try:
            agg = []
            commentNum = f'{i:04}'
    #         url= links.loc[i]['url']#+'?api_key='+apiKey
            url = 'https://api.regulations.gov/v4/comments/ATF-2021-0002-'+str(i)+'?api_key='+apiKey
            response =  requests.get(url, verify=True)
            content = response.content
            if 'The document with the specified ID could not be found' in str(content):
                print('no doc found for comment number: '+ str(i))
                print(url)
                break

            attributes = json.loads(content.decode("utf-8"))['data']['attributes']
            firstName = attributes['firstName']
            lastName = attributes['lastName']
            modifyDate = attributes['modifyDate']
            postedDate = attributes['postedDate']    
            postmarkDate = attributes['postmarkDate']
            receiveDate = attributes['receiveDate']
            email = attributes['email']
            city = attributes['city']
            stateProvinceRegion = attributes['stateProvinceRegion']
            zip = attributes['zip']
            trackingNbr = attributes['trackingNbr']
            commentUrl = url
            comment = attributes['comment']
            agg.append({'idx':i,'firstName':firstName,'lastName':lastName, 'modifyDate':modifyDate,\
                        'postedDate':postedDate,'postmarkDate':postmarkDate, 'receiveDate':receiveDate,\
                        'email':email,'city':city, 'stateProvinceRegion':stateProvinceRegion,\
                        'zip':zip,'trackingNbr':trackingNbr, 'commentUrl':commentUrl,\
                        'comment':comment})
            commentsDF = pd.DataFrame(agg)

            commentsDF.to_csv(r'brace_comment.csv', mode='a', header=False)

            commentsDF['comment']= commentsDF['comment'].apply(cleanhtml)
            commentsDF['nltkSentiment'] = commentsDF['comment'].apply(getSentiments)
            commentsDF['sentimentPosition'] = commentsDF['nltkSentiment'].apply(sentimentPosition)
            commentsDF.to_csv(r'brace_sentiment.csv', mode='a', header=False)
            display(commentsDF[['nltkSentiment', 'sentimentPosition','comment']])

            print('commentNum:'+str(i))
            lastCommentProcessed = i
    #         print(url)
        except:
            print('could not fetch: '+str(i))
            print(url)
            e = sys.exc_info()[0]
            print(e)
            e = sys.exc_info()[1]
            break
    print ('endCycle: ' +  datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"))
#     print('waiting 30 minutes and trying again with comment num:'+str(lastCommentProcessed))
#     sleep(900)
#     print('15 more minutes before next cycle')
#     sleep(600)
#     print('5 minutes before next cycle')
#     sleep(240)
    print('1 minute to go before next cycle')
    sleep(60)
    DLandScore(lastCommentProcessed)


In [ ]:
DLandScore(nextComment)